# Check GPU and Import Packages

In [ ]:
!nvidia-smi

Sun Dec  3 00:42:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install ./diffusers
!pip install -U -r /content/diffusers/examples/text_to_image/requirements.txt
!pip install wandb

#Config the Accelerate and Hugging Face
More about Accelerate Can Be Found [Here](https://huggingface.co/docs/accelerate/index).

In [ ]:
!accelerate config default --mixed_precision fp16

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
import os

os.environ['MODEL_NAME'] = f'kerianheyi/CS245-fine-tunedSD14200_14600_14122'
os.environ['DATASET_NAME'] = f'jytjyt05/t_to_m7'
os.environ['OUTPUT_DIR'] = f'kerianheyi/CS245-fine-tunedSD14600_15000_14122'

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import wandb
# from google.colab import drive
# drive.mount('/content/drive/')

#Run the Training

In [ ]:
'''Main Script for Fine-Tuning Stable Diffusion'''
!accelerate launch diffusers/examples/text_to_image/train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --caption_column="caption" \
  --use_ema \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=400 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir=$OUTPUT_DIR \
  --checkpointing_steps=400 \
  --validation_prompts="A melSpectrogram for piano solo in C-Minor" \
  --push_to_hub

2023-12-03 00:16:09.658684: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 00:16:09.658742: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 00:16:09.658784: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 00:16:10.913595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/03/2023 00:16:13 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'sample_max_value', 'thresholding', 'variance_type', 'clip_sample_rang

#Run inference on our best model

In [ ]:
from diffusers import DiffusionPipeline
import torch

model_path="kerianheYi/CS245-fine-tunedSD13800_14200_14122"
pipeline = DiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipeline.to("cuda")
prompt = "A melSpectrogram for piano solo in minor"
for i in range(5,50):
  image = pipeline(prompt).images[0]
  image.save("Minor"+str(i+1)+".png")